In [58]:
import numpy as np

In [87]:
pauliZ = np.array([[1,0],[0,-1]])
I = np.array([[1,0],[0,1]])
N=2

In [93]:
S0Z = pauliZ
for i in range(N-1):
    S0Z = np.kron(S0Z,I)
print(S0Z)

[[ 1  0  0  0]
 [ 0  1  0  0]
 [ 0  0 -1  0]
 [ 0  0  0 -1]]


In [94]:
S1Zgen = np.full((N,2,2),I)
S1Zgen[0] = pauliZ
print(S1Zgen)
S1Z = S1Zgen[0]
for i in range(N-1):
    S1Z = np.kron(S1Z,S1Zgen[i+1])
print(S1Z)

[[[ 1  0]
  [ 0 -1]]

 [[ 1  0]
  [ 0  1]]]
[[ 1  0  0  0]
 [ 0  1  0  0]
 [ 0  0 -1  0]
 [ 0  0  0 -1]]
